In [ ]:
# Cell 1: Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Try to import windrose, use alternative if not available
try:
    from windrose import WindroseAxes
    has_windrose = True
except ImportError:
    has_windrose = False
    print("Windrose not installed, using alternative plots")

print("All libraries imported successfully")

In [ ]:
# Cell 2: Data Loading
# Replace with your actual data file path
df = pd.read_csv('data/raw/benin_data.csv')  # Update with actual filename

print("Dataset Shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nData Types:")
print(df.dtypes)

In [ ]:
# Cell 3: Summary Statistics & Missing Value Report
print("=== SUMMARY STATISTICS ===")
print(df.describe())

print("\n=== MISSING VALUE REPORT ===")
missing_report = df.isna().sum()
missing_percent = (df.isna().sum() / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_report,
    'Missing Percentage': missing_percent
})
print(missing_df)

# Flag columns with >5% nulls
high_missing = missing_df[missing_df['Missing Percentage'] > 5]
print("\nColumns with >5% missing values:")
print(high_missing)

In [ ]:
# Cell 4: Outlier Detection with Z-scores
numeric_cols = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']
z_score_threshold = 3

print("=== OUTLIER DETECTION (Z-scores > 3) ===")
outlier_report = {}

for col in numeric_cols:
    if col in df.columns:
        z_scores = np.abs(stats.zscore(df[col].dropna()))
        outlier_count = (z_scores > z_score_threshold).sum()
        outlier_percent = (outlier_count / len(df[col].dropna())) * 100
        outlier_report[col] = {
            'outlier_count': outlier_count,
            'outlier_percent': outlier_percent
        }
        print(f"{col}: {outlier_count} outliers ({outlier_percent:.2f}%)")

outlier_df = pd.DataFrame(outlier_report).T

In [ ]:
# Cell 5: Data Cleaning
print("=== DATA CLEANING ===")
df_clean = df.copy()

# Create cleaning flag
df_clean['cleaning_flag'] = 'original'

# Handle missing values in key columns
key_columns = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust', 'Tamb', 'RH']

for col in key_columns:
    if col in df_clean.columns:
        missing_before = df_clean[col].isna().sum()
        df_clean[col].fillna(df_clean[col].median(), inplace=True)
        missing_after = df_clean[col].isna().sum()
        print(f"{col}: Filled {missing_before} missing values with median")

# Handle outliers using IQR method
for col in numeric_cols:
    if col in df_clean.columns:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers_mask = (df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)
        outlier_count = outliers_mask.sum()
        
        if outlier_count > 0:
            df_clean.loc[outliers_mask, 'cleaning_flag'] = 'outlier_removed'
            print(f"{col}: Flagged {outlier_count} outliers")

print("Cleaning completed!")

In [ ]:
# Cell 6: Export Cleaned Data
# Ensure data/ is in .gitignore before exporting!
df_clean.to_csv('data/processed/benin_clean.csv', index=False)
print("Cleaned data exported to data/processed/benin_clean.csv")

# Verify .gitignore has data/
with open('.gitignore', 'r') as f:
    gitignore_content = f.read()
    if 'data/' in gitignore_content:
        print("✓ data/ is in .gitignore - CSV won't be committed")
    else:
        print("⚠ WARNING: data/ not in .gitignore - add it!")

In [ ]:
# Cell 7: Time Series Analysis
if 'Timestamp' in df_clean.columns:
    df_clean['Timestamp'] = pd.to_datetime(df_clean['Timestamp'])
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Time Series Analysis', fontsize=16)
    
    # GHI over time
    axes[0,0].plot(df_clean['Timestamp'], df_clean['GHI'], alpha=0.7)
    axes[0,0].set_title('GHI vs Time')
    axes[0,0].set_ylabel('GHI')
    
    # DNI over time
    axes[0,1].plot(df_clean['Timestamp'], df_clean['DNI'], alpha=0.7, color='orange')
    axes[0,1].set_title('DNI vs Time')
    axes[0,1].set_ylabel('DNI')
    
    # DHI over time
    axes[1,0].plot(df_clean['Timestamp'], df_clean['DHI'], alpha=0.7, color='green')
    axes[1,0].set_title('DHI vs Time')
    axes[1,0].set_ylabel('DHI')
    
    # Temperature over time
    axes[1,1].plot(df_clean['Timestamp'], df_clean['Tamb'], alpha=0.7, color='red')
    axes[1,1].set_title('Temperature vs Time')
    axes[1,1].set_ylabel('Temperature')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Cell 8: Cleaning Impact Analysis
if 'cleaning_flag' in df_clean.columns:
    cleaning_impact = df_clean.groupby('cleaning_flag')[['ModA', 'ModB']].mean()
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    cleaning_impact[['ModA', 'ModB']].plot(kind='bar', ax=axes[0])
    axes[0].set_title('Average ModA & ModB Pre/Post Cleaning')
    axes[0].set_ylabel('Average Value')
    
    # Count of records in each cleaning category
    df_clean['cleaning_flag'].value_counts().plot(kind='pie', ax=axes[1], autopct='%1.1f%%')
    axes[1].set_title('Data Cleaning Distribution')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Cell 9: Correlation Analysis
correlation_cols = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'Tamb', 'RH', 'WS', 'WSgust']
available_cols = [col for col in correlation_cols if col in df_clean.columns]

plt.figure(figsize=(10, 8))
corr_matrix = df_clean[available_cols].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, square=True)
plt.title('Correlation Heatmap')
plt.tight_layout()
plt.show()

In [ ]:
# Cell 10: Scatter Plot Relationships
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# WS vs GHI
if all(col in df_clean.columns for col in ['WS', 'GHI']):
    axes[0,0].scatter(df_clean['WS'], df_clean['GHI'], alpha=0.5)
    axes[0,0].set_xlabel('Wind Speed (WS)')
    axes[0,0].set_ylabel('GHI')
    axes[0,0].set_title('WS vs GHI')

# RH vs Tamb
if all(col in df_clean.columns for col in ['RH', 'Tamb']):
    axes[0,1].scatter(df_clean['RH'], df_clean['Tamb'], alpha=0.5, color='green')
    axes[0,1].set_xlabel('Relative Humidity (RH)')
    axes[0,1].set_ylabel('Temperature (Tamb)')
    axes[0,1].set_title('RH vs Temperature')

# RH vs GHI
if all(col in df_clean.columns for col in ['RH', 'GHI']):
    axes[1,0].scatter(df_clean['RH'], df_clean['GHI'], alpha=0.5, color='red')
    axes[1,0].set_xlabel('Relative Humidity (RH)')
    axes[1,0].set_ylabel('GHI')
    axes[1,0].set_title('RH vs GHI')

# WSgust vs GHI
if all(col in df_clean.columns for col in ['WSgust', 'GHI']):
    axes[1,1].scatter(df_clean['WSgust'], df_clean['GHI'], alpha=0.5, color='purple')
    axes[1,1].set_xlabel('Wind Gust (WSgust)')
    axes[1,1].set_ylabel('GHI')
    axes[1,1].set_title('WSgust vs GHI')

plt.tight_layout()
plt.show()

In [ ]:
# Cell 11: Wind Analysis (Wind Rose)
if all(col in df_clean.columns for col in ['WS', 'WD']) and has_windrose:
    fig = plt.figure(figsize=(10, 8))
    ax = WindroseAxes.from_ax(fig=fig)
    ax.bar(df_clean['WD'].dropna(), df_clean['WS'].dropna(), normed=True, opening=0.8, edgecolor='white')
    ax.set_legend()
    plt.title('Wind Rose - Wind Speed vs Direction')
    plt.show()
elif all(col in df_clean.columns for col in ['WS', 'WD']):
    # Alternative wind analysis
    plt.figure(figsize=(10, 6))
    plt.scatter(df_clean['WD'], df_clean['WS'], alpha=0.5)
    plt.xlabel('Wind Direction (WD)')
    plt.ylabel('Wind Speed (WS)')
    plt.title('Wind Speed vs Direction')
    plt.show()

In [ ]:
# Cell 12: Distribution Analysis
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# GHI Histogram
if 'GHI' in df_clean.columns:
    axes[0,0].hist(df_clean['GHI'], bins=50, alpha=0.7, edgecolor='black')
    axes[0,0].set_xlabel('GHI')
    axes[0,0].set_ylabel('Frequency')
    axes[0,0].set_title('GHI Distribution')

# WS Histogram
if 'WS' in df_clean.columns:
    axes[0,1].hist(df_clean['WS'], bins=50, alpha=0.7, edgecolor='black', color='orange')
    axes[0,1].set_xlabel('Wind Speed (WS)')
    axes[0,1].set_ylabel('Frequency')
    axes[0,1].set_title('Wind Speed Distribution')

# Temperature Histogram
if 'Tamb' in df_clean.columns:
    axes[1,0].hist(df_clean['Tamb'], bins=50, alpha=0.7, edgecolor='black', color='red')
    axes[1,0].set_xlabel('Temperature (Tamb)')
    axes[1,0].set_ylabel('Frequency')
    axes[1,0].set_title('Temperature Distribution')

# RH Histogram
if 'RH' in df_clean.columns:
    axes[1,1].hist(df_clean['RH'], bins=50, alpha=0.7, edgecolor='black', color='green')
    axes[1,1].set_xlabel('Relative Humidity (RH)')
    axes[1,1].set_ylabel('Frequency')
    axes[1,1].set_title('Relative Humidity Distribution')

plt.tight_layout()
plt.show()

In [ ]:
# Cell 13: Bubble Chart
if all(col in df_clean.columns for col in ['GHI', 'Tamb', 'RH']):
    plt.figure(figsize=(10, 6))
    scatter = plt.scatter(df_clean['GHI'], df_clean['Tamb'], 
                         s=df_clean['RH']/2, alpha=0.6, 
                         c=df_clean['RH'], cmap='viridis')
    plt.colorbar(scatter, label='Relative Humidity (RH)')
    plt.xlabel('GHI')
    plt.ylabel('Temperature (Tamb)')
    plt.title('GHI vs Temperature (Bubble size = RH)')
    plt.show()

In [ ]:
# Cell 14: Summary Insights
print("=== EDA SUMMARY INSIGHTS ===")
print(f"Dataset size: {df_clean.shape}")
print(f"Original records: {len(df_clean[df_clean['cleaning_flag'] == 'original'])}")
print(f"Records with outliers: {len(df_clean[df_clean['cleaning_flag'] == 'outlier_removed'])}")

if 'GHI' in df_clean.columns:
    print(f"\nGHI Statistics:")
    print(f"  Mean: {df_clean['GHI'].mean():.2f}")
    print(f"  Max: {df_clean['GHI'].max():.2f}")
    print(f"  Min: {df_clean['GHI'].min():.2f}")

print("\n=== ACTIONABLE INSIGHTS ===")
# Add your specific insights here based on the analysis